# Test bench starter code

This code is just starter code such that anyone can copy-paste own sruff into it and check various things over time, i.e:

* Accuracy of different churn definitions
* Data drift
* Model performance over time

The options are endless. This test bench will allow more robust insights into how well models or design decisions might work.

In [6]:
import gc
from datetime import timedelta
import glob
import pandas as pd

# Load all train data

In [2]:
file_pattern = '/kaggle/input/neo-bank-non-sub-churn-prediction/train_*.parquet' 
all_files = glob.glob(file_pattern)
all_files

['/kaggle/input/neo-bank-non-sub-churn-prediction/train_2008.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2020.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2019.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2010.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2011.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2016.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2017.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2015.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2009.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2021.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2014.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2022.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2018.parquet',
 '/kaggle/input/neo-bank-non-sub-churn-prediction/train_2023.parquet',
 '/kag

In [3]:
df = pd.concat([pd.read_parquet(file) for file in all_files], ignore_index=True)
df

,Id,customer_id,interest_rate,name,country,date_of_birth,address,date,atm_transfer_in,atm_transfer_out,...,bank_transfer_out_volume,crypto_in_volume,crypto_out_volume,complaints,touchpoints,csat_scores,tenure,from_competitor,job,churn_due_to_fraud
0,1,1,3.5,Yolanda Parker,Lithuania,1954-07-10,"1929 Erin Lights Suite 709\nLake Michaelburgh,...",2008-01-17,0,0,...,5844.193394,393.145010,220.900654,0,[],"{'appointment': None, 'email': None, 'phone': ...",0,False,Amenity horticulturist,False
1,6,1,3.5,Yolanda Parker,Lithuania,1954-07-10,"1929 Erin Lights Suite 709\nLake Michaelburgh,...",2008-01-18,0,0,...,4986.370246,390.005729,221.147296,0,[],"{'appointment': None, 'email': None, 'phone': ...",1,False,Amenity horticulturist,False
2,16,1,3.5,Yolanda Parker,Lithuania,1954-07-10,"1929 Erin Lights Suite 709\nLake Michaelburgh,...",2008-01-19,0,0,...,7451.373471,393.209108,260.510535,0,[],"{'appointment': None, 'email': None, 'phone': ...",2,False,Amenity horticulturist,False
3,31,1,3.5,Yolanda Parker,Lithuania,1954-07-10,"1929 Erin Lights Suite 709\nLake Michaelburgh,...",2008-01-20,0,0,...,6548.352211,394.078294,203.214128,0,[],"{'appointment': None, 'email': None, 'phone': ...",3,False,Amenity horticulturist,False
4,50,1,3.5,Yolanda Parker,Lithuania,1954-07-10,"1929 Erin Lights Suite 709\nLake Michaelburgh,...",2008-01-21,0,0,...,6129.342716,395.174604,207.894467,0,"[whatsapp, email, email]","{'appointment': None, 'email': None, 'phone': ...",4,False,Amenity horticulturist,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926053,848170,13664,0.0,Brittany Potter,Poland,1964-10-06,8978 Michele Crossroad Apt. 755\nEast Jasonpor...,2013-12-31,4,0,...,89.955215,45.546970,12.188301,0,[whatsapp],"{'appointment': None, 'email': None, 'phone': ...",0,False,"Development worker, international aid",False
3926054,848171,13665,0.0,Angela Garcia,Italy,1953-03-26,"3196 Anna Trail Apt. 801\nWest Christopher, CO...",2013-12-31,0,0,...,104.285413,56.098345,12.743148,0,[],"{'appointment': None, 'email': None, 'phone': ...",0,False,Claims inspector/assessor,False
3926055,848172,13666,0.0,Carol Robertson,Poland,1976-01-29,"5607 James Lock Apt. 409\nClarkborough, UT 416...",2013-12-31,0,0,...,102.762738,46.569205,13.111458,0,[],"{'appointment': None, 'email': None, 'phone': ...",0,False,"Designer, exhibition/display",False
3926056,848173,13667,0.0,Barry Powell,Austria,1959-04-20,"259 Hernandez Glens\nJacobsonland, OK 56372 Au...",2013-12-31,0,0,...,10.174721,48.731619,14.354938,0,[],"{'appointment': None, 'email': None, 'phone': ...",0,True,Audiological scientist,False


# Define loop

In [30]:
def metric_of_choice(*args):
    """Add any evaluation of your choice here."""
    return 0

In [31]:
df = df.sort_values("date")
min_date = df["date"].min()
max_date = df["date"].max()

window_size_days = 365
window = timedelta(days=window_size_days)
step_size = 365
step = timedelta(days=step_size)

current_train_end = min_date + window

print(f"Min date: {min_date}, max date: {max_date}. Current current_train_end is {current_train_end}.")

Min date: 2008-01-17 00:00:00, max date: 2023-12-31 00:00:00. Current current_train_end is 2009-01-16 00:00:00.


In [32]:
results = []

while current_train_end <= max_date:
    print(f"Point in time is {current_train_end}")
    current_train_start = current_train_end - window
        
    train_data = df.loc[
        (df["date"] >= current_train_start) &
        (df["date"] < current_train_end)
    ]
    current_test_start, current_test_end = current_train_end, current_train_end + step
    
    test_data = df[(df["date"] >= current_test_start) & (df["date"] < current_test_end)]
    
    # Check if there's enough test data, otherwise break the loop
    if test_data.empty:
        break
    
    print(f"Training from {current_train_start} to {current_train_end}")
    print(f"Testing from {current_test_start} to {current_test_end}")


    # store results
    result_df = pd.DataFrame(
        {
            "train_data_start": [current_train_start],
            "train_data_end": [current_train_end],
            "test_data_start": [current_test_start],
            "test_data_end": [current_test_end],
            "step_size": step_size,
            "window_size_days": window_size_days,
            "metric": metric_of_choice()
        }
    )
    results.append(result_df)


    # Move the training and test windows forward
    current_train_end += step

    del train_data, test_data, result_df
    _ = gc.collect()

Point in time is 2009-01-16 00:00:00
Training from 2008-01-17 00:00:00 to 2009-01-16 00:00:00
Testing from 2009-01-16 00:00:00 to 2010-01-16 00:00:00
Point in time is 2010-01-16 00:00:00
Training from 2009-01-16 00:00:00 to 2010-01-16 00:00:00
Testing from 2010-01-16 00:00:00 to 2011-01-16 00:00:00
Point in time is 2011-01-16 00:00:00
Training from 2010-01-16 00:00:00 to 2011-01-16 00:00:00
Testing from 2011-01-16 00:00:00 to 2012-01-16 00:00:00
Point in time is 2012-01-16 00:00:00
Training from 2011-01-16 00:00:00 to 2012-01-16 00:00:00
Testing from 2012-01-16 00:00:00 to 2013-01-15 00:00:00
Point in time is 2013-01-15 00:00:00
Training from 2012-01-16 00:00:00 to 2013-01-15 00:00:00
Testing from 2013-01-15 00:00:00 to 2014-01-15 00:00:00
Point in time is 2014-01-15 00:00:00
Training from 2013-01-15 00:00:00 to 2014-01-15 00:00:00
Testing from 2014-01-15 00:00:00 to 2015-01-15 00:00:00
Point in time is 2015-01-15 00:00:00
Training from 2014-01-15 00:00:00 to 2015-01-15 00:00:00
Testin

# Collecting results

In this part we just union all results and can begin doing deeper analysis.

In [33]:
results_df = pd.concat(results)
results_df

,train_data_start,train_data_end,test_data_start,test_data_end,step_size,window_size_days,metric
0,2008-01-17,2009-01-16,2009-01-16,2010-01-16,365,365,0
0,2009-01-16,2010-01-16,2010-01-16,2011-01-16,365,365,0
0,2010-01-16,2011-01-16,2011-01-16,2012-01-16,365,365,0
0,2011-01-16,2012-01-16,2012-01-16,2013-01-15,365,365,0
0,2012-01-16,2013-01-15,2013-01-15,2014-01-15,365,365,0
0,2013-01-15,2014-01-15,2014-01-15,2015-01-15,365,365,0
0,2014-01-15,2015-01-15,2015-01-15,2016-01-15,365,365,0
0,2015-01-15,2016-01-15,2016-01-15,2017-01-14,365,365,0
0,2016-01-15,2017-01-14,2017-01-14,2018-01-14,365,365,0
0,2017-01-14,2018-01-14,2018-01-14,2019-01-14,365,365,0


In [34]:
results_df["metric"].describe()

count    15.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: metric, dtype: float64